In [2]:
from tqdm import tqdm
import pandas as pd
import json

In [3]:
with open("/datadrive/pavan/repos/synthetic_reasoning/data/all_pairs.jsonl") as f:
    all_pairs = json.load(f)
print(len(all_pairs))

1111110


In [17]:
question = []
answer = []
solution = []
algorithm = []
length = []
text = []
c = 0
for i in tqdm(range(len(all_pairs))):
    for k in all_pairs[i]:
        if k.endswith('_sort'):
            q = ' '.join(map(str, all_pairs[i]['unsorted']))
            a = ' '.join(map(str, all_pairs[i]['sorted']))
            s = '<|T|> '+' <|T|> '.join([' '.join(map(str, step)) for step in all_pairs[i][k]])
            algo = k.split('_')[0]
            question.append(q)
            answer.append(a)
            solution.append(s)
            algorithm.append(algo)
            length.append(len(all_pairs[i]['sorted']))
            text.append("<|Q|> "+q+" <|S|> "+algo+" "+s+" <|A|> "+a+" <|E|>")

100%|██████████| 1111110/1111110 [01:10<00:00, 15823.03it/s]


In [20]:
df = pd.DataFrame()
df['question'] = question
df['answer'] = answer
df['solution'] = solution
df['algorithm'] = algorithm
df['length'] = length
df['text'] = text
df.head()

,question,answer,solution,algorithm,length,text
0,0,0,<|T|> 0,bubble,1,<|Q|> 0 <|S|> bubble <|T|> 0 <|A|> 0 <|E|>
1,0,0,<|T|> 0,selection,1,<|Q|> 0 <|S|> selection <|T|> 0 <|A|> 0 <|E|>
2,0,0,<|T|> 0,insertion,1,<|Q|> 0 <|S|> insertion <|T|> 0 <|A|> 0 <|E|>
3,0,0,<|T|> 0,quick,1,<|Q|> 0 <|S|> quick <|T|> 0 <|A|> 0 <|E|>
4,0,0,<|T|> 0,heap,1,<|Q|> 0 <|S|> heap <|T|> 0 <|A|> 0 <|E|>


In [23]:
df['length'].value_counts().sort_index()

length
1         80
2        800
3       8000
4      80000
5     800000
6    8000000
Name: count, dtype: int64

In [4]:
max_len = 6
algos = ["greedy", "dp", "recursive", "brute_force"]

base_path = "/datadrive/pavan/az_storage/repos/synthetic_reasoning/data/solution_paths/pairs_len"

data_dict = {}

for i in tqdm(range(1, max_len + 1)):
    data_dict[i] = {}
    for algo in algos:
        data_dict[i][algo] = []
        path = f"{base_path}{i}_{algo}.pkl"
        with open(path, "rb") as f:
            data = pickle.load(f)
            data_dict[i][algo] = data

100%|██████████| 6/6 [01:40<00:00, 16.73s/it]


In [7]:
for l in data_dict:
    for algo in data_dict[l]:
        # Get the lengths of each inner list
        lengths = [len(inner_list) for inner_list in data_dict[l][algo]]
        # Calculate the mean length
        mean_length = sum(lengths) / len(lengths) if lengths else 0
        print(algo, ": ", mean_length, end=", ")
    print()


greedy :  1.0, dp :  1.0, recursive :  1.0, brute_force :  2.0, 
greedy :  1.45, dp :  2.0, recursive :  2.0, brute_force :  2.45, 
greedy :  2.065, dp :  3.0, recursive :  3.0, brute_force :  4.25, 
greedy :  2.7625, dp :  3.0, recursive :  4.0, brute_force :  12.3005, 
greedy :  3.50917, dp :  4.0, recursive :  5.0, brute_force :  54.758, 
greedy :  4.288345, 

dp :  4.0, recursive :  6.0, brute_force :  316.302245, 


In [118]:
data = [
    (l, algo, inner_list[0], inner_list[-1], inner_list[1:-1] if len(inner_list) > 2 else [inner_list[0]])
    for l in data_dict
    for algo in data_dict[l]
    for inner_list in data_dict[l][algo]
]

In [119]:
df = pd.DataFrame(data, columns=["length", "algo", "question", "answer", "solution"])

In [120]:
df['length'].value_counts().sort_index()

length
1         40
2        400
3       4000
4      40000
5     400000
6    4000000
Name: count, dtype: int64

In [26]:
# Shuffle and filter by length
df4 = df[df['length'] == 4].sample(frac=1, random_state=42)
df5 = df[df['length'] == 5].sample(frac=1, random_state=42)
df6 = df[df['length'] == 6].sample(frac=1, random_state=42)

# --------- Phase 1: Supervised Pretraining ---------

# Use equal number from each length (up to 40k)
n_per_length = min(len(df4), len(df5), len(df6), 40000)

# Sample equal data
df4_ = df4.iloc[:n_per_length]
df5_ = df5.iloc[:n_per_length]
df6_ = df6.iloc[:n_per_length]

# Combine and shuffle
pretrain_base = pd.concat([df4_, df5_, df6_]).sample(frac=1, random_state=1).reset_index(drop=True)

# Add the lengths 1, 2, 3 to pretrain
df123 = df[df['length'].isin([1, 2, 3])].copy()
df123["split"] = "pretrain"

# Split pretrain_base into train/val/test
n_total = len(pretrain_base)
n_train = int(0.9 * n_total)
n_val = int(0.05 * n_total)
n_test = n_total - (n_train + n_val)

pretrain_df = pretrain_base.iloc[:n_train].copy()
val_df = pretrain_base.iloc[n_train:n_train + n_val].copy()
test_df = pretrain_base.iloc[n_train + n_val:].copy()

pretrain_df["split"] = "pretrain"
val_df["split"] = "val"
test_df["split"] = "test"

# --------- Phase 2: RL Fine-tuning (length=6 only) ---------

# Use remaining `length=6` data
rl_data = df6.iloc[n_per_length:].copy()
rl_finetune_df = rl_data.sample(n=100000, random_state=42).copy()
rl_finetune_df["split"] = "rl_finetune"

# --------- Final Combined Dataset ---------

final_df = pd.concat([pretrain_df, val_df, test_df, rl_finetune_df, df123]).reset_index(drop=True)

# Sanity checks
print(final_df["split"].value_counts())
print(final_df.groupby(["split", "length"]).size())

# Optional: save
# final_df.to_csv("two_phase_split_with_short.csv", index=False)


split
pretrain       116880
rl_finetune    100000
val              6000
test             6000
Name: count, dtype: int64
split        length
pretrain     1             80
             2            800
             3           8000
             4          35981
             5          36025
             6          35994
rl_finetune  6         100000
test         4           2060
             5           1949
             6           1991
val          4           1959
             5           2026
             6           2015
dtype: int64


In [27]:
#group by algorithms and split
final_df.groupby(['split', 'algorithm']).size().unstack()

algorithm,bubble,cycle,gnome,heap,insertion,quick,selection,shell
split,,,,,,,,
pretrain,14674,14714,14676,14529,14662,14613,14421,14591
rl_finetune,12435,12480,12374,12724,12511,12394,12532,12550
test,752,786,767,735,726,734,742,758
val,753,720,794,785,746,727,707,768


In [28]:
final_df.head()

,question,answer,solution,algorithm,length,text,split
0,3 0 7 6 6,0 3 6 6 7,<|T|> 3 0 7 6 6 <|T|> 3 0 6 6 7 <|T|> 0 3 6 6 7,shell,5,<|Q|> 3 0 7 6 6 <|S|> shell <|T|> 3 0 7 6 6 <|...,pretrain
1,2 3 8 0,0 2 3 8,<|T|> 2 3 8 0 <|T|> 2 2 8 0 <|T|> 2 2 3 0 <|T|...,cycle,4,<|Q|> 2 3 8 0 <|S|> cycle <|T|> 2 3 8 0 <|T|> ...,pretrain
2,2 5 7 7 1,1 2 5 7 7,<|T|> 2 5 7 7 1 <|T|> 1 2 5 7 7,insertion,5,<|Q|> 2 5 7 7 1 <|S|> insertion <|T|> 2 5 7 7 ...,pretrain
3,4 4 5 0,0 4 4 5,<|T|> 4 4 5 0 <|T|> 0 4 4 5,insertion,4,<|Q|> 4 4 5 0 <|S|> insertion <|T|> 4 4 5 0 <|...,pretrain
4,4 4 7 0 3 7,0 3 4 4 7 7,<|T|> 4 4 7 0 3 7 <|T|> 4 4 0 7 3 7 <|T|> 4 0 ...,gnome,6,<|Q|> 4 4 7 0 3 7 <|S|> gnome <|T|> 4 4 7 0 3 ...,pretrain


In [ ]:
from datasets import Dataset, DatasetDict

# Group by "split" and create DatasetDict
split_datasets = {
    split: Dataset.from_pandas(sub_df.drop(columns=["split"]))
    for split, sub_df in final_df.groupby("split")
}

dataset_dict = DatasetDict(split_datasets)

# Push to Hugging Face Hub
dataset_dict.push_to_hub("Pavankalyan/syn-reasoning", token="")


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/117 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/100 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/6 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/6 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Pavankalyan/syn-reasoning/commit/1160202f288e07c50eb3b9be4bc8d61d76f6ebdc', commit_message='Upload dataset', commit_description='', oid='1160202f288e07c50eb3b9be4bc8d61d76f6ebdc', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/Pavankalyan/syn-reasoning', endpoint='https://huggingface.co', repo_type='dataset', repo_id='Pavankalyan/syn-reasoning'), pr_revision=None, pr_num=None)

In [30]:
final_df['algorithm'].value_counts().sort_index()

algorithm
bubble       28614
cycle        28700
gnome        28611
heap         28773
insertion    28645
quick        28468
selection    28402
shell        28667
Name: count, dtype: int64